In [15]:
from distutils.log import info
import datetime
import pandas as pd
import numpy as np
from SQL import SQL_TabellenLadenBearbeiten as SQL

##TODO: SAFE BACKUP LT22 BEFORE ANYTHING

class DatenAgregieren():

    def sapLt22DatenBerechnen(dflt22):
        dfStammdaten = SQL.sql_Stammdaten()
        dfOrders = SQL.sql_datenTabelleLaden('business_depotDEBYKN-DepotDEBYKNOrders')
        dfMitarbeiter = SQL.sql_datenTabelleLaden('Mitarbeiter')
        dflt22.set_index('Transfer Order Number', inplace=True)

        # Weil ich nunmal ein Excel Idiot bi

        dflt22.columns = ['MaterialNumber','C','D','E','F','G','H','I','J','K','L','M','MitarbeiterCreateTO','MitarbeiterConfirmTO','P','Q','R','S','T','DestStorType','DestBin','W','X','Y','Z','AA','AB','AC''ToNumber']
        dflt22['Ziel'] = dflt22['DestStorType'].str[:2]
        dflt22['Quelle'] = dflt22['E'].str[:2]
        dflt22['Pick Datum'] = dflt22['L'].dt.strftime('%m/%d/%y')
        #dflt22['K'] = dflt22['K'].astype(str)
        dflt22['MaterialNumber'] = dflt22['MaterialNumber'].astype(str)
        dflt22['K'] = pd.to_datetime(dflt22['K'], format='%H:%M:%S')
        dflt22['Pick Zeit'] = dflt22['K']
        dflt22['L'] = pd.to_datetime(dflt22['L'])
        dflt22['SuperDepot'] = dflt22['DestBin'].isin(dfOrders['SapOrderNumber'])
        dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]
        dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
        dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]   
        def f_CS(row):
            try:
                if row.UnitOfMeasure == 'CS':          
                    return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
            except:
                return np.nan
        def f_PAL(row):
            try:
                if row.UnitOfMeasure == 'D97':
                    return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
            except:
                return np.nan
        def f_OUT(row):
            try:
                if row.UnitOfMeasure == 'OUT':
                    return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
            except:
                return np.nan
        dfStammdaten['OUT'] = dfStammdaten.apply(f_OUT,axis=1)
        dfStammdaten['CS'] = dfStammdaten.apply(f_CS,axis=1)
        dfStammdaten['PAL'] = dfStammdaten.apply(f_PAL,axis=1)
        

        dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
        dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
        dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

        # print('pickschleife')
        # #Schleife erst Picks Berechnen -> Dann Umlagerungen ermitteln
        for each in dflt22.index:
            if dflt22.loc[each, 'E'] == 'TN1' and dflt22.loc[each, 'DestStorType'] == '916' and dflt22.loc[each, 'DestStorType'] == '916':
                dflt22.loc[each, 'PICKS'] = dflt22.loc[each, 'H'] 
                dflt22.loc[each, 'Picks OUT'] = dflt22.loc[each, 'H'] 
                dflt22.loc[each, 'Pick Art'] = 'Stange'
            if dflt22.loc[each, 'E'] == 'SN1' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN2' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN3' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN4' and dflt22.loc[each, 'DestStorType'] == '916':
                dflt22.loc[each, 'PICKS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'CS']
                dflt22.loc[each, 'Picks CS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'CS']
                dflt22.loc[each, 'Pick Art'] = 'Karton'
            if dflt22.loc[each, 'E'] == 'BS3' and dflt22.loc[each, 'DestStorType'] == '916':
                dflt22.loc[each, 'PICKS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'PAL']
                dflt22.loc[each, 'PICKS PAL'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'PAL']
                dflt22.loc[each, 'Pick Art'] = 'Palette'
            #Umlagerungen Ermitteln 
            if dflt22.loc[each, 'E'] == 'BS3' and dflt22.loc[each, 'Ziel'] == 'SN':
                dflt22.loc[each, 'Umlagerung'] = 1
                dflt22.loc[each, 'Art'] = 'Karton'
            if dflt22.loc[each, 'Quelle'] == 'RS' and dflt22.loc[each, 'Ziel'] == 'SN':
                dflt22.loc[each, 'Umlagerung'] = 1
                dflt22.loc[each, 'Art'] = 'Karton'
            if dflt22.loc[each, 'Quelle'] == 'SN' and dflt22.loc[each, 'Ziel'] == 'TN':
                dflt22.loc[each, 'Umlagerung'] = 1
                dflt22.loc[each, 'Art'] = 'Gebinde'

        # change dfMitabeiter OneID to string 
        dfMitarbeiter['One ID'] = dfMitarbeiter['One ID'].astype(str)
        dflt22['MitarbeiterConfirmTO'] = dflt22['MitarbeiterConfirmTO'].astype(str)
        #dfMitarbeiter['One ID'] = dfMitarbeiter['One ID'].apply(lambda x: 999999999 if type(x) == str else x)

        ## Füge den Mitarbeiter hinzu
        # if the value of dfMitarbeiter[OneID] exists in dflt22[MitarbeiterConfirmTO] then add the value of dfMitarbeiter[Mitarbeiter] to dflt22[MitarbeiterConfirmTO]
        dflt22 = pd.merge(dflt22, dfMitarbeiter,left_on='MitarbeiterConfirmTO', right_on='One ID',how='left')
        #fill none with 'Unbekannt'
        dflt22['Name'] = dflt22['Name'].fillna('Unbekannt')
        dflt22['LieferscheinErhalten'] = dfOrders['SapOrderNumber'].apply(lambda x: dfOrders.loc[dfOrders['SapOrderNumber'] == x]['CreatedTimestamp'].iloc[0])
        dflt22['PartnerNo'] = dfOrders['SapOrderNumber'].apply(lambda x: dfOrders.loc[dfOrders['SapOrderNumber'] == x]['PartnerNo'].iloc[0])
        dflt22['SuperDepot'] = dflt22['DestBin'].isin(dfOrders['SapOrderNumber'])
        #df['Truck Kennzeichen'] = df['SapOrderNumber'].apply(lambda x: df1.loc[df1['SapOrderNumber'] == x]['UnloadingListIdentifier'].iloc[0])
        try:
            dflt22['Lieferschein'] = dflt22['DestBin'].apply(lambda x: dflt22.loc[dflt22['SuperDepot'] == True]['DestBin'].iloc[0])
        except:
            pass
        
        # save to parquet
        #dflt22.to_parquet('lt22.parquet')

        #PartnerNo
        return dflt22

In [16]:
df03 = pd.read_excel('03okt.xlsx')
df04 = pd.read_excel('04okt.xlsx')
df05 = pd.read_excel('05okt.xlsx')

In [17]:
df03bea = DatenAgregieren.sapLt22DatenBerechnen(df03)
df04bea = DatenAgregieren.sapLt22DatenBerechnen(df04)
df05bea = DatenAgregieren.sapLt22DatenBerechnen(df05)

In [18]:
df04bea.head(5)

,MaterialNumber,C,D,E,F,G,H,I,J,K,...,Picks OUT,One ID,Name,Funktion,Unternehmen,Fachbereich,Status,LieferscheinErhalten,PartnerNo,Lieferschein
0,10129175,NaN,NaN,SN1,R-09-01-09,NaN,124.2,KG,LUCK ORRED 30.00 6 LM GEW PI,1900-01-01 14:58:11,...,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 05:52:18.966519+00:00,None,3202159290
1,10129201,NaN,NaN,SN1,R-10-01-17,NaN,124.2,KG,SAMS HALFZ 30.00 6 LM GEW PI,1900-01-01 14:23:03,...,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:15.694118+00:00,None,3202159290
2,10129137,NaN,NaN,BS3,G7Q07,1.599890e+17,124.2,KG,PALL AMEBL 30.00 6 LM GEW PI,1900-01-01 15:31:16,...,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:16.774748+00:00,None,3202159290
3,10158284,NaN,NaN,BS3,G7A10,6.590314e+17,42.0,CS,PALL RED 31/248 KRE SQ GEW PUIB,1900-01-01 01:18:54,...,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:43.688397+00:00,None,3202159290
4,10138228,NaN,NaN,GS3,GR-DOCK,1.590141e+17,7.0,CS,HB RNDED 20/200 KRE SQ GEW PUIBIF,1900-01-01 04:18:09,...,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 05:52:09.259484+00:00,None,3202159290


In [19]:
df03bea.head(5)

,MaterialNumber,C,D,E,F,G,H,I,J,K,...,Umlagerung,Art,One ID,Name,Funktion,Unternehmen,Fachbereich,Status,LieferscheinErhalten,PartnerNo
0,10140667,NaN,NaN,902,4200175167,NaN,208.32,TH,PALL AUTRD 31/248 KRE SQ GEW,1900-01-01 23:13:34,...,NaN,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 05:52:18.966519+00:00,None
1,10140667,NaN,NaN,902,4200175167,NaN,208.32,TH,PALL AUTRD 31/248 KRE SQ GEW,1900-01-01 23:15:33,...,NaN,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:15.694118+00:00,None
2,10139128,NaN,NaN,902,4200175167,NaN,194.88,TH,LUCK RED 29/232 KRE SQ GEW PUIB,1900-01-01 23:18:42,...,NaN,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:16.774748+00:00,None
3,10128174,NaN,NaN,BS3,G7C02,3.403130e+17,35.00,CS,HB CLBL 80.00 4 CC GEW,1900-01-01 22:28:11,...,1.0,Karton,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 06:07:43.688397+00:00,None
4,10158873,NaN,NaN,902,4200175167,NaN,194.88,TH,LUCK STRRE 29/232 KRE SQ GEW CHW1,1900-01-01 23:14:18,...,NaN,NaN,NaN,Unbekannt,NaN,NaN,NaN,NaN,2022-06-21 05:52:09.259484+00:00,None


In [21]:
df05bea.columns

Index(['MaterialNumber', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'MitarbeiterCreateTO', 'MitarbeiterConfirmTO', 'P', 'Q', 'R', 'S', 'T',
       'DestStorType', 'DestBin', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'ACToNumber',
       'Ziel', 'Quelle', 'Pick Datum', 'Pick Zeit', 'SuperDepot', 'CS', 'PAL',
       'OUT', 'Umlagerung', 'Art', 'PICKS', 'Picks OUT', 'Pick Art',
       'Picks CS', 'PICKS PAL', 'One ID', 'Name', 'Funktion', 'Unternehmen',
       'Fachbereich', 'Status', 'LieferscheinErhalten', 'PartnerNo',
       'Lieferschein'],
      dtype='object')

In [22]:
df03bea.tonub

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59],
           dtype='int64')